In [3]:
import csv
import random
import time
import json

In [4]:
# 读取 /home/zjusst/qms/webarena/result_stage_1_explore/flitered_triplets.csv
with open("/home/zjusst/qms/webarena/result_stage_1_explore/flitered_triplets.csv", "r") as f:
    reader = csv.reader(f)
    # 跳过第一行
    next(reader)
    triplets = [row for row in reader]
len(triplets)

1025

In [5]:
out_degree_dict = {}
for triplet in triplets:
    if triplet[2] not in out_degree_dict:
        out_degree_dict[triplet[2]] = 0
    out_degree_dict[triplet[2]] += 1

# 转为 list，由低到高排序
out_degree_list = sorted(out_degree_dict.items(), key=lambda x: x[1])


In [6]:
flitered_out_degree_list = []
for item in out_degree_list:
    real_url = eval(item[0])[3]
    # 如果 real_url 的  "/" 大于 3 个，则加入 flitered_out_degree_list
    if real_url.count("/") > 3 and real_url.startswith("http://reddit.com/"):
        flitered_out_degree_list.append(item)

len(flitered_out_degree_list)

66

In [7]:
def random_select_url_and_a11y(flitered_out_degree_list):
    # 种子为时间
    random.seed(time.time())
    record = flitered_out_degree_list[random.randint(0, len(flitered_out_degree_list) - 1)]
    file_name, traj_index, pos, real_url = eval(record[0])
    with open(f"/home/zjusst/qms/webarena/result_stage_1_explore/trajs/{file_name}", "r") as f:
        trajs = json.load(f)
    traj = trajs[traj_index]
    if pos == 0:
        a11y = traj["a11y_before"]
    else:
        a11y = traj["a11y_after"]
    return real_url, a11y

In [8]:
def random_persona(personas):
    random.seed(time.time())
    return personas[random.randint(0, len(personas) - 1)]

In [9]:
personas = []
with open("/home/zjusst/qms/persona-hub/data/elite_personas_10000.jsonl", "r") as f:
    for line in f:
        persona = json.loads(line)
        personas.append(persona['persona'])

In [10]:
random_persona(personas)

'A cultural psychologist who specializes in understanding the psychology of zero-sum cultures, particularly in their relationship with envy and mistrust. They are interested in exploring how these cultures view the world as a limited resource and how envy and mistrust are often prevalent in these societies. They are also curious about how cultures respond to and struggle against these emotions, and how they can be overcome or managed in order to promote a more equitable and just society. They may have a particular interest in understanding how envy and mistrust can be used as tools for social control and how they can be overcome in order to foster a more positive and inclusive culture.'

In [16]:
real_url, a11y = random_select_url_and_a11y(flitered_out_degree_list)
real_url, a11y[:100]

('http://reddit.com/submit/AskReddit',
 "Tab 0 (current): Create submission\n\n[9157] RootWebArea 'Create submission' focused: True\n\t[9197] Hea")